## Helper functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from others.implementations import *
import torch
from model import *

In [3]:
def generate_model(shallow_hidden_channels, deep_hidden_channels):
    return Sequential(
        Conv2d(3, shallow_hidden_channels, kernel_size=2, stride=2, padding=0),
        ReLU(),
        Conv2d(shallow_hidden_channels, deep_hidden_channels, kernel_size=2, stride=2, padding=0),
        ReLU(),
        NearestUpsampling(scale_factor=2),
        Conv2d(deep_hidden_channels, shallow_hidden_channels, kernel_size=3, stride=1, padding=1),
        ReLU(),
        NearestUpsampling(scale_factor=2),
        Conv2d(shallow_hidden_channels, 3, kernel_size=3, stride=1, padding=1),
        Sigmoid()
        )

def test_and_train(batch_size, hidden_channels, lr, momentum, nesterov, epochs):
    network = generate_model(shallow_hidden_channels=hidden_channels[0], deep_hidden_channels=hidden_channels[1])
    optimizer = SGD(network.param(), lr=lr, momentum=momentum, nesterov=nesterov)

    # Instantiate model and replace network and optimizer
    m = Model()
    m.model = network
    m.optimizer = optimizer
    m.batch_size = batch_size

    # Train
    m.train(s1, s2, epochs)

    return compute_psnr(m.predict(t1), t2), m

def sample(tensor1, tensor2, k):
    perm = torch.randperm(tensor1.size(0))
    idx = perm[:k]
    return tensor1[idx], tensor2[idx]

## Loading data

In [4]:
path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'
noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1, noisy_imgs_2 = noisy_imgs_1.double(), noisy_imgs_2.double() / 255.0
test, truth = torch.load(path_val)
test, truth = test.double(), truth.double() / 255.0

## Parameters to tune

In [5]:
# Channels parameter
shallow_hidden_channels = [8, 16]
deep_hidden_channels = [16, 32]
hidden_channels = [(shallow_channel, deep_channel) for shallow_channel in shallow_hidden_channels for deep_channel in deep_hidden_channels if shallow_channel <= deep_channel]

# Optimizer parameters
lrs = np.logspace(-7, -1, 5)
momentums = [0.9]
nesterovs = [False]

# Batch sizes
batch_sizes = [5, 100]

## Training

In [6]:
s1, s2 = sample(noisy_imgs_1, noisy_imgs_2, 500)
t1, t2 = sample(test, truth, 500)

In [7]:
epochs = 5

results = dict()
for lr in lrs:
    for momentum in momentums:
        for nesterov in nesterovs:
            for hidden_channel in hidden_channels:
                for batch_size in batch_sizes:
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000'
                    print("\nSTARTING TRAINING FOR:", description)
                    error, m = test_and_train(batch_size, hidden_channel, lr, momentum, nesterov, epochs)
                    description = f'lr{lr}_HiddenChannels{hidden_channel}_Batch{batch_size}_Epochs{epochs}_Momentum{momentum}_Nesterov{nesterov}_Sample1000_{error}'
                    results[description] = m
                    print(f'PSNR: {error}')


STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 16)_Batch5_Epochs5_Momentum0.9_NesterovFalse_Sample1000
EPOCH 0 --- LOSS 0.7828586412970643
EPOCH 1 --- LOSS 0.7820689497500568
EPOCH 2 --- LOSS 0.7811976207702701
EPOCH 3 --- LOSS 0.780320688660314
EPOCH 4 --- LOSS 0.7794381196297844
FINAL LOSS 0.7785498534817992
PSNR: 1.095138087891373

STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 16)_Batch100_Epochs5_Momentum0.9_NesterovFalse_Sample1000
EPOCH 0 --- LOSS 0.2900275163709924
EPOCH 1 --- LOSS 0.29002330845348906
EPOCH 2 --- LOSS 0.2900142935060818
EPOCH 3 --- LOSS 0.29000244298572225
EPOCH 4 --- LOSS 0.28998892038677015
FINAL LOSS 0.28997441331810986
PSNR: 5.433103700853621

STARTING TRAINING FOR: lr1e-07_HiddenChannels(8, 32)_Batch5_Epochs5_Momentum0.9_NesterovFalse_Sample1000
EPOCH 0 --- LOSS 0.23013926012451016
EPOCH 1 --- LOSS 0.22929733408101746
EPOCH 2 --- LOSS 0.228360370915308
EPOCH 3 --- LOSS 0.2274107353475524
EPOCH 4 --- LOSS 0.22644832412819563
FINAL LOSS 0.22547310781618

KeyboardInterrupt: 

## Final results